# Required vs Optional Fields

So far, all the fields we defined in our Pydantic model were required.

As an analogy, how do we make function arguments optional in Python?

We provide the argument a default value.

The same approach is used by Pydantic.

We make a field optional, by simply providing the field definition with a default value.

There are a few ways of doing this, we'll explore one way here - and explore other ways later in the course.

In [1]:
from pydantic import BaseModel

class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int

In this model, `center` is an **optional** field, that will default to `(0, 0)` if not provided in the data we are deserializing. on the other hand, since `radius` does not have a default defined, it is a **required** field.

We can also see this by inspecting the model fields:

In [2]:
Circle.model_fields

{'center': FieldInfo(annotation=tuple[int, int], required=False, default=(0, 0)),
 'radius': FieldInfo(annotation=int, required=True)}

Note how `center` has the `required=False` property, while `radius` has `required=True`.

We can now create instances of `Circle` without providing a value for `center`:

In [3]:
Circle(radius=1)

Circle(center=(0, 0), radius=1)

As you can see, no validation exception was raised, and we have the proper default value in place.

This works the same way for all the other deserializtion methods too:

In [4]:
data = {"radius": 1}
data_json = '{"radius": 1}'

In [5]:
Circle.model_validate(data)

Circle(center=(0, 0), radius=1)

In [6]:
Circle.model_validate_json(data_json)

Circle(center=(0, 0), radius=1)

Of course, we can provide a value for center too:

In [7]:
Circle(center=(1, 1), radius=2)

Circle(center=(1, 1), radius=2)

We have to be quite careful about one thing when specifying a default value for a field.

When we provide a field value, Pydantic validates that value before putting it into our model instance. However, Pydantic's default behavior does not validate default values!

That kind of makes sense - after all, we are writing the model definition, so we should be able to only provide a valid default value in our model definition.

So, here's the issue:

In [8]:
class Model(BaseModel):
    field: int = "Python"

As you can see, the provided default is totally inconsistent for an `int` type, and yet this still works:

In [9]:
Model()

Model(field='Python')

So, be careful when providing default values, the onus is on us, as developers, to make sure we provided consistent defaults.

Pydantic does offer us a way to force default data validations - but it does mean a little of extra compute time to validate that default value every time it is needed. Something we can avoid if we write correct code. We'll see later how to enable default validations on a model.

Usually in Python, including dataclasses, we have to be extra careful when we assign a default that is a mutable object.

let's look at an example of this with a regular Python function:

By the way, writing a function that behaves this way - modifies it's input and returns it, is a terrible coding technique - I'm just using this to illustrate a point.

We can call this function with our own list:

In [10]:
from time import time

def extend_list(user_list: list = []):
    user_list.append(int(time()))
    return user_list

In [11]:
my_times = []
extend_list(my_times)
my_times

[1701540599]

In [12]:
my_times = extend_list()
my_times

[1701540599]

And this seems to have worked.

But what about this?

In [13]:
my_new_times = extend_list()

Our expectation might be that `my_new_times` just contains one element, whatever the epoch time was when we called `extend_list()`.

In [14]:
my_new_times

[1701540599, 1701540599]

Huh??

This is the issue I was telling you about. When we defined a default for function arguments, these values are calculated and stored with the function itself - they are not re-created every time the function is called. In other words, that default value is going to be **shared** amogst all the function calls.

Something similar happens with dataclases. In dataclasses, we get around the problem by a mechanism that is called a default factory. Instead of providing a mutable object as a default value, we provide a function that will get called each time a dataclass instance is created, and that function will therefore return a **new** default object.

Pydantic is very similar, in that it offers this default factory idea (which we'll cover later).

However, it goes one step further, and actually allows us to simply define mutable objects as defaults. 

When Pydantic sees this, it will actually create a deep copy of the mutable object every time a new model instance is created.